In [96]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
import warnings
warnings.filterwarnings("ignore")

In [97]:
pd.set_option('display.max_columns', None)

In [98]:
file = 'stellar_samples.csv'

In [99]:
k2 = pd.read_csv(file)

In [100]:
k2

,EPIC,Campaign,CDPP1,CDPP1_5,CDPP2,CDPP2_5,CDPP3,CDPP4,CDPP5,CDPP6,CDPP7,CDPP8,CDPP9,CDPP10,SpT,RUWE,Ncomp,Rad,U_Rad,u_Rad,Teff,U_Teff,logg,U_logg,Mass,U_Mass,FeH,U_FeH,limb1,limb2,d_mag
0,201121245,1,257.151,236.598,226.563,209.841,191.452,174.727,171.493,155.131,149.860,144.238,141.405,133.911,G5,1.174929,0.0,4.415,0.348,0.329,4814,138,3.283,0.150,1.373,0.610,-0.100,0.235,0.6212,0.1129,0.0
1,201122454,1,268.578,248.491,225.499,214.708,198.934,183.430,183.657,170.828,173.404,180.913,183.539,172.220,K7,7.138988,0.0,0.950,0.078,0.071,4046,138,4.532,0.150,1.123,0.521,-0.161,0.235,0.4639,0.2589,0.0
2,201122521,1,486.386,450.572,368.257,355.254,328.753,295.503,264.617,253.877,242.677,221.716,213.211,211.158,G8,0.947940,0.0,2.522,0.287,0.247,5144,138,3.652,0.150,1.052,0.511,-0.375,0.235,0.4664,0.2249,0.0
3,201123619,1,639.118,587.075,533.719,479.562,471.031,425.428,373.952,361.346,343.016,318.207,315.377,314.689,M1,1.127277,0.0,0.528,0.017,0.017,3676,138,4.713,0.043,0.525,0.042,-0.038,0.235,0.4651,0.2700,0.0
4,201124136,1,102.980,96.421,87.393,88.527,81.895,78.648,71.903,72.715,72.086,71.319,68.591,72.448,K7,10.175707,0.0,0.602,0.046,0.041,4028,138,4.638,0.150,0.576,0.260,-0.475,0.235,0.3568,0.3354,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241334,211730267,18,208.273,206.536,177.306,175.085,170.118,140.380,153.043,160.229,139.191,150.403,131.543,137.804,K4,0.966075,0.0,0.995,0.068,0.062,5383,138,4.490,0.151,1.110,0.499,0.213,0.198,0.6767,0.0697,0.0
241335,211830266,18,846.281,825.627,736.583,626.123,610.549,520.170,467.402,431.880,402.149,381.403,367.534,354.297,M3,0.977467,0.0,1.598,0.129,0.115,4578,141,3.577,0.154,0.352,0.168,-0.214,0.250,0.3284,0.3566,0.0
241336,211914998,18,100.992,92.874,93.439,93.967,98.588,108.393,109.383,113.219,110.807,112.347,116.787,114.714,G4,0.976495,0.0,0.898,0.059,0.053,5539,138,4.420,0.151,0.779,0.352,-0.141,0.198,0.4415,0.2396,0.0
241337,211962097,18,295.027,358.410,294.420,299.896,309.960,312.412,273.927,267.456,274.259,275.153,270.212,311.431,F2,0.980242,0.0,2.174,0.144,0.132,6187,138,3.986,0.151,1.677,0.730,-0.194,0.198,0.3430,0.2667,0.0


In [101]:
k2 = k2.rename(columns={'EPIC': 'EPIC_ID'})

In [102]:
k2all = pd.read_csv('https://www.dropbox.com/scl/fi/ba9f1x2it1ukqar7xjlt8/k2all.csv?rlkey=a17ije9hsdle1t2ygr9v2izi5&st=rkx8dt2m&dl=1')

In [103]:
k2all

,EPIC_ID,Campaign,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,E_Dist,Teff,e_Teff,logg,e_logg,feh,e_feh,Lum,e_Lum,Rad,e_Rad,Mass,e_Mass,RUWE,non_single_star,Gmag,e_Gmag,BPmag,e_BPmag,RPmag,e_RPmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,BP-RP,BP-G,G-RP,RP-J,J-H,H-K,extinctionJ,extinctionH,extinctionK,absJ,e_absJ,absH,e_absH,absK,e_absK,BC,e_BC
0,201121245,1,3598701493287932544,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,6.073486,4902,143,3.416,0.066,-0.257,0.217,9.064502,0.684049,4.177,0.245,1.661,0.134,1.175,0,11.102720,0.002764,11.604373,0.002830,10.439149,0.003798,9.620,0.026,9.100,0.023,8.981,0.019,1.165,0.502,0.664,0.819,0.520,0.119,0.000,0.000,0.000,1.041,0.040,0.521,0.036,0.400,0.033,1.942,0.078
1,201122521,1,3598719154193454976,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,97.159910,5094,149,3.784,0.094,-0.288,0.217,3.620656,0.499565,2.453,0.252,1.332,0.113,0.948,0,14.746737,0.002763,15.206782,0.003263,14.127944,0.003942,13.402,0.026,12.901,0.027,12.872,0.035,1.078,0.460,0.619,0.726,0.501,0.029,0.000,0.000,0.000,2.078,0.113,1.575,0.113,1.546,0.115,1.833,0.078
2,201123619,1,3598719497790839040,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,1.719025,3729,109,4.758,0.031,-0.633,0.217,0.047647,0.002202,0.500,0.015,0.519,0.019,1.127,0,15.143640,0.002773,16.153950,0.003832,14.147896,0.003945,12.855,0.024,12.285,0.023,12.060,0.024,2.006,1.010,0.996,1.293,0.570,0.225,0.000,0.000,0.000,6.337,0.031,5.771,0.031,5.546,0.030,2.505,0.041
3,201124275,1,3598743206010226560,2MASS J11445381-0555420,176.224230,0.370010,-5.928365,0.230490,532.702515,5.769959,7.512146,5942,174,4.706,0.079,-1.779,0.217,0.506598,0.046664,0.674,0.052,0.833,0.068,1.046,0,14.176920,0.002766,14.471922,0.002914,13.705976,0.003858,13.148,0.028,12.827,0.023,12.796,0.034,0.764,0.295,0.471,0.558,0.321,0.031,0.024,0.014,0.009,4.489,0.039,4.183,0.036,4.153,0.045,1.331,0.087
4,201126062,1,3598723620959410816,2MASS J11465755-0553007,176.739796,1.016033,-5.883488,0.543972,244.279556,3.241867,3.339600,3776,110,4.684,0.028,-0.831,0.217,0.075336,0.003976,0.577,0.018,0.593,0.022,1.634,0,15.078585,0.002775,16.049982,0.003648,14.098021,0.003892,12.867,0.022,12.223,0.023,12.006,0.021,1.952,0.971,0.981,1.231,0.644,0.217,0.000,0.000,0.000,5.930,0.036,5.285,0.037,5.070,0.036,2.482,0.051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257482,212179420,18,678425788468626816,2MASS J08304609+2343144,127.692046,0.256701,23.720657,0.220915,929.733521,13.142090,14.060424,6219,182,4.124,0.065,-0.420,0.217,3.623429,0.252327,1.665,0.109,1.338,0.108,0.987,0,13.175423,0.002765,13.433455,0.002836,12.755074,0.003842,12.321,0.023,12.040,0.022,11.981,0.026,0.679,0.258,0.420,0.434,0.281,0.059,0.024,0.014,0.009,2.456,0.040,2.183,0.039,2.130,0.041,1.207,0.066
257483,212179600,18,678250004049172480,2MASS J08274958+2343309,126.956631,0.220381,23.725236,0.169956,1289.393310,26.359250,30.733280,4922,144,3.315,0.074,-0.170,0.217,13.001449,1.277752,4.907,0.377,1.819,0.149,0.936,0,12.659062,0.002763,13.158696,0.002855,12.003857,0.003792,11.169,0.022,10.664,0.020,10.579,0.017,1.156,0.500,0.655,0.835,0.505,0.085,0.000,0.000,0.000,0.617,0.054,0.111,0.054,0.025,0.053,1.939,0.091
257484,212179688,18,678241070517213056,2MASS J08263429+2343389,126.642887,0.320074,23.727465,0.213690,736.999084,8.799316,10.122925,5993,176,4.159,0.064,-0.167,0.217,2.875102,0.232305,1.560,0.102,1.262,0.103,0.924,0,12.899457,0.002767,13.192869,0.002824,12.445556,0.003797,11.932,0.020,11.659,0.028,11.614,0.025,0.748,0.293,0.454,0.514,0.273,0.045,0.024,0.014,0.009,2.572,0.034,2.308,0.040,2.268,0.037,1.315,0.076
257485,212179696,18,678230212839882112,2MASS J08270421+2343393,126.767550,0.392856,23.727629,0.291679,225.309784,1.121964,1.219345,4879

In [104]:
k2.columns

Index(['EPIC_ID', 'Campaign', 'CDPP1', 'CDPP1_5', 'CDPP2', 'CDPP2_5', 'CDPP3',
       'CDPP4', 'CDPP5', 'CDPP6', 'CDPP7', 'CDPP8', 'CDPP9', 'CDPP10', 'SpT',
       'RUWE', 'Ncomp', 'Rad', 'U_Rad', 'u_Rad', 'Teff', 'U_Teff', 'logg',
       'U_logg', 'Mass', 'U_Mass', 'FeH', 'U_FeH', 'limb1', 'limb2', 'd_mag'],
      dtype='object')

In [105]:
new_columns = [col for col in k2all.columns if col not in k2.columns]

In [106]:
new_columns

['GaiaDR3',
 '2MASS',
 'RAJ2000',
 'e_RAJ2000',
 'DEJ2000',
 'e_DEJ2000',
 'Dist',
 'e_Dist',
 'E_Dist',
 'e_Teff',
 'e_logg',
 'feh',
 'e_feh',
 'Lum',
 'e_Lum',
 'e_Rad',
 'e_Mass',
 'non_single_star',
 'Gmag',
 'e_Gmag',
 'BPmag',
 'e_BPmag',
 'RPmag',
 'e_RPmag',
 'Jmag',
 'e_Jmag',
 'Hmag',
 'e_Hmag',
 'Kmag',
 'e_Kmag',
 'BP-RP',
 'BP-G',
 'G-RP',
 'RP-J',
 'J-H',
 'H-K',
 'extinctionJ',
 'extinctionH',
 'extinctionK',
 'absJ',
 'e_absJ',
 'absH',
 'e_absH',
 'absK',
 'e_absK',
 'BC',
 'e_BC']

In [107]:
k2_updated = k2.merge(k2all, on='EPIC_ID', how='left')

In [108]:
k2_updated

,EPIC_ID,Campaign_x,CDPP1,CDPP1_5,CDPP2,CDPP2_5,CDPP3,CDPP4,CDPP5,CDPP6,CDPP7,CDPP8,CDPP9,CDPP10,SpT,RUWE_x,Ncomp,Rad_x,U_Rad,u_Rad,Teff_x,U_Teff,logg_x,U_logg,Mass_x,U_Mass,FeH,U_FeH,limb1,limb2,d_mag,Campaign_y,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,E_Dist,Teff_y,e_Teff,logg_y,e_logg,feh,e_feh,Lum,e_Lum,Rad_y,e_Rad,Mass_y,e_Mass,RUWE_y,non_single_star,Gmag,e_Gmag,BPmag,e_BPmag,RPmag,e_RPmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,BP-RP,BP-G,G-RP,RP-J,J-H,H-K,extinctionJ,extinctionH,extinctionK,absJ,e_absJ,absH,e_absH,absK,e_absK,BC,e_BC
0,201121245,1,257.151,236.598,226.563,209.841,191.452,174.727,171.493,155.131,149.860,144.238,141.405,133.911,G5,1.174929,0.0,4.415,0.348,0.329,4814,138,3.283,0.150,1.373,0.610,-0.100,0.235,0.6212,0.1129,0.0,1.0,3.598701e+18,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,6.073486,4902.0,143.0,3.416,0.066,-0.257,0.217,9.064502,0.684049,4.177,0.245,1.661,0.134,1.175,0.0,11.102720,0.002764,11.604373,0.002830,10.439149,0.003798,9.620,0.026,9.100,0.023,8.981,0.019,1.165,0.502,0.664,0.819,0.520,0.119,0.000,0.000,0.000,1.041,0.040,0.521,0.036,0.400,0.033,1.942,0.078
1,201122454,1,268.578,248.491,225.499,214.708,198.934,183.430,183.657,170.828,173.404,180.913,183.539,172.220,K7,7.138988,0.0,0.950,0.078,0.071,4046,138,4.532,0.150,1.123,0.521,-0.161,0.235,0.4639,0.2589,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201122521,1,486.386,450.572,368.257,355.254,328.753,295.503,264.617,253.877,242.677,221.716,213.211,211.158,G8,0.947940,0.0,2.522,0.287,0.247,5144,138,3.652,0.150,1.052,0.511,-0.375,0.235,0.4664,0.2249,0.0,1.0,3.598719e+18,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,97.159910,5094.0,149.0,3.784,0.094,-0.288,0.217,3.620656,0.499565,2.453,0.252,1.332,0.113,0.948,0.0,14.746737,0.002763,15.206782,0.003263,14.127944,0.003942,13.402,0.026,12.901,0.027,12.872,0.035,1.078,0.460,0.619,0.726,0.501,0.029,0.000,0.000,0.000,2.078,0.113,1.575,0.113,1.546,0.115,1.833,0.078
3,201123619,1,639.118,587.075,533.719,479.562,471.031,425.428,373.952,361.346,343.016,318.207,315.377,314.689,M1,1.127277,0.0,0.528,0.017,0.017,3676,138,4.713,0.043,0.525,0.042,-0.038,0.235,0.4651,0.2700,0.0,1.0,3.598719e+18,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,1.719025,3729.0,109.0,4.758,0.031,-0.633,0.217,0.047647,0.002202,0.500,0.015,0.519,0.019,1.127,0.0,15.143640,0.002773,16.153950,0.003832,14.147896,0.003945,12.855,0.024,12.285,0.023,12.060,0.024,2.006,1.010,0.996,1.293,0.570,0.225,0.000,0.000,0.000,6.337,0.031,5.771,0.031,5.546,0.030,2.505,0.041
4,201124136,1,102.980,96.421,87.393,88.527,81.895,78.648,71.903,72.715,72.086,71.319,68.591,72.448,K7,10.175707,0.0,0.602,0.046,0.041,4028,138,4.638,0.150,0.576,0.260,-0.475,0.235,0.3568,0.3354,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262792,211730267,18,208.273,206.536,177.306,175.085,170.118,140.380,153.043,160.229,139.191,150.403,131.543,137.804,K4,0.966075,0.0,0.995,0.068,0.062,5383,138,4.490,0.151,1.110,0.499,0.213,0.198,0.6767,0.0697,0.0,18.0,6.101815e+17,2MASS J08474528+1619440,131.938701,0.288863,16.328929,0.227015,506.285309,4.147034,4.576446,5627.0,165.0,4.458,0.061,-0.129,0.217,0.813536,0.069009,0.953,0.069,0.932,0.075,0.979,0.0,13.467436,0.002760,13.819328,0.002866,

In [109]:
k2_updated['GaiaDR3']

0         3.598701e+18
1                  NaN
2         3.598719e+18
3         3.598719e+18
4                  NaN
              ...     
262792    6.101815e+17
262793    6.572477e+17
262794    6.604007e+17
262795    6.607804e+17
262796             NaN
Name: GaiaDR3, Length: 262797, dtype: float64

In [110]:
k2_updated.columns

Index(['EPIC_ID', 'Campaign_x', 'CDPP1', 'CDPP1_5', 'CDPP2', 'CDPP2_5',
       'CDPP3', 'CDPP4', 'CDPP5', 'CDPP6', 'CDPP7', 'CDPP8', 'CDPP9', 'CDPP10',
       'SpT', 'RUWE_x', 'Ncomp', 'Rad_x', 'U_Rad', 'u_Rad', 'Teff_x', 'U_Teff',
       'logg_x', 'U_logg', 'Mass_x', 'U_Mass', 'FeH', 'U_FeH', 'limb1',
       'limb2', 'd_mag', 'Campaign_y', 'GaiaDR3', '2MASS', 'RAJ2000',
       'e_RAJ2000', 'DEJ2000', 'e_DEJ2000', 'Dist', 'e_Dist', 'E_Dist',
       'Teff_y', 'e_Teff', 'logg_y', 'e_logg', 'feh', 'e_feh', 'Lum', 'e_Lum',
       'Rad_y', 'e_Rad', 'Mass_y', 'e_Mass', 'RUWE_y', 'non_single_star',
       'Gmag', 'e_Gmag', 'BPmag', 'e_BPmag', 'RPmag', 'e_RPmag', 'Jmag',
       'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag', 'BP-RP', 'BP-G', 'G-RP',
       'RP-J', 'J-H', 'H-K', 'extinctionJ', 'extinctionH', 'extinctionK',
       'absJ', 'e_absJ', 'absH', 'e_absH', 'absK', 'e_absK', 'BC', 'e_BC'],
      dtype='object')

In [111]:
# Cast GaiaDR3 IDs as strings
k2_updated['GaiaDR3'] = k2_updated['GaiaDR3'].astype('|S').str.decode('utf-8')

In [112]:
# Read in APOGEE DR17 data
data = Table.read('https://portal.nersc.gov/project/cosmo/data/sdss/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStarLite-dr17-synspec_rev1.fits', format='fits', hdu=1)
del data['PARAMFLAG', 'X_H', 'X_H_ERR', 'X_M', 'X_M_ERR', 'ELEM_CHI2', 'ELEMFRAC', 'X_H_SPEC', 'X_M_SPEC']
df = data.to_pandas()

In [113]:
df.columns

Index(['APOGEE_ID', 'TELESCOPE', 'FIELD', 'ALT_ID', 'RA', 'DEC', 'GLON',
       'GLAT', 'J', 'J_ERR',
       ...
       'NI_FE_ERR', 'NI_FE_FLAG', 'CU_FE', 'CU_FE_SPEC', 'CU_FE_ERR',
       'CU_FE_FLAG', 'CE_FE', 'CE_FE_SPEC', 'CE_FE_ERR', 'CE_FE_FLAG'],
      dtype='object', length=182)

In [114]:
# Make some quality cuts
dfq = df[(df['SNR']>100)]

In [115]:
print(df.columns.tolist())

['APOGEE_ID', 'TELESCOPE', 'FIELD', 'ALT_ID', 'RA', 'DEC', 'GLON', 'GLAT', 'J', 'J_ERR', 'H', 'H_ERR', 'K', 'K_ERR', 'AK_TARG', 'AK_TARG_METHOD', 'AK_WISE', 'SFD_EBV', 'APOGEE_TARGET1', 'APOGEE_TARGET2', 'APOGEE2_TARGET1', 'APOGEE2_TARGET2', 'APOGEE2_TARGET3', 'APOGEE2_TARGET4', 'TARGFLAGS', 'SURVEY', 'PROGRAMNAME', 'NVISITS', 'SNR', 'SNREV', 'STARFLAG', 'STARFLAGS', 'ANDFLAG', 'ANDFLAGS', 'VHELIO_AVG', 'VSCATTER', 'VERR', 'RV_TEFF', 'RV_LOGG', 'RV_FEH', 'RV_ALPHA', 'RV_CARB', 'RV_CHI2', 'RV_CCFWHM', 'RV_AUTOFWHM', 'RV_FLAG', 'N_COMPONENTS', 'MEANFIB', 'SIGFIB', 'MIN_H', 'MAX_H', 'MIN_JK', 'MAX_JK', 'GAIAEDR3_SOURCE_ID', 'GAIAEDR3_PARALLAX', 'GAIAEDR3_PARALLAX_ERROR', 'GAIAEDR3_PMRA', 'GAIAEDR3_PMRA_ERROR', 'GAIAEDR3_PMDEC', 'GAIAEDR3_PMDEC_ERROR', 'GAIAEDR3_PHOT_G_MEAN_MAG', 'GAIAEDR3_PHOT_BP_MEAN_MAG', 'GAIAEDR3_PHOT_RP_MEAN_MAG', 'GAIAEDR3_DR2_RADIAL_VELOCITY', 'GAIAEDR3_DR2_RADIAL_VELOCITY_ERROR', 'GAIAEDR3_R_MED_GEO', 'GAIAEDR3_R_LO_GEO', 'GAIAEDR3_R_HI_GEO', 'GAIAEDR3_R_MED_PHOTO

In [116]:
# Drop targets with bad stellar parameters
dfq.dropna(subset=['TEFF', 'TEFF_ERR', 'LOGG', 'LOGG_ERR', 'M_H', 'M_H_ERR', 'FE_H', 'FE_H_ERR'],inplace=True)
dfq = dfq.sort_values(by='SNR',ascending=False)

In [117]:
# Keep highest SNR spectrum if multiples exist
dfq.drop_duplicates(subset='GAIAEDR3_SOURCE_ID',inplace=True)
dfq.reset_index(inplace=True,drop=True)

In [118]:
print(dfq.columns.tolist())

['APOGEE_ID', 'TELESCOPE', 'FIELD', 'ALT_ID', 'RA', 'DEC', 'GLON', 'GLAT', 'J', 'J_ERR', 'H', 'H_ERR', 'K', 'K_ERR', 'AK_TARG', 'AK_TARG_METHOD', 'AK_WISE', 'SFD_EBV', 'APOGEE_TARGET1', 'APOGEE_TARGET2', 'APOGEE2_TARGET1', 'APOGEE2_TARGET2', 'APOGEE2_TARGET3', 'APOGEE2_TARGET4', 'TARGFLAGS', 'SURVEY', 'PROGRAMNAME', 'NVISITS', 'SNR', 'SNREV', 'STARFLAG', 'STARFLAGS', 'ANDFLAG', 'ANDFLAGS', 'VHELIO_AVG', 'VSCATTER', 'VERR', 'RV_TEFF', 'RV_LOGG', 'RV_FEH', 'RV_ALPHA', 'RV_CARB', 'RV_CHI2', 'RV_CCFWHM', 'RV_AUTOFWHM', 'RV_FLAG', 'N_COMPONENTS', 'MEANFIB', 'SIGFIB', 'MIN_H', 'MAX_H', 'MIN_JK', 'MAX_JK', 'GAIAEDR3_SOURCE_ID', 'GAIAEDR3_PARALLAX', 'GAIAEDR3_PARALLAX_ERROR', 'GAIAEDR3_PMRA', 'GAIAEDR3_PMRA_ERROR', 'GAIAEDR3_PMDEC', 'GAIAEDR3_PMDEC_ERROR', 'GAIAEDR3_PHOT_G_MEAN_MAG', 'GAIAEDR3_PHOT_BP_MEAN_MAG', 'GAIAEDR3_PHOT_RP_MEAN_MAG', 'GAIAEDR3_DR2_RADIAL_VELOCITY', 'GAIAEDR3_DR2_RADIAL_VELOCITY_ERROR', 'GAIAEDR3_R_MED_GEO', 'GAIAEDR3_R_LO_GEO', 'GAIAEDR3_R_HI_GEO', 'GAIAEDR3_R_MED_PHOTO

In [119]:
dfq.head()

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,H,H_ERR,K,K_ERR,AK_TARG,AK_TARG_METHOD,AK_WISE,SFD_EBV,APOGEE_TARGET1,APOGEE_TARGET2,APOGEE2_TARGET1,APOGEE2_TARGET2,APOGEE2_TARGET3,APOGEE2_TARGET4,TARGFLAGS,SURVEY,PROGRAMNAME,NVISITS,SNR,SNREV,STARFLAG,STARFLAGS,ANDFLAG,ANDFLAGS,VHELIO_AVG,VSCATTER,VERR,RV_TEFF,RV_LOGG,RV_FEH,RV_ALPHA,RV_CARB,RV_CHI2,RV_CCFWHM,RV_AUTOFWHM,RV_FLAG,N_COMPONENTS,MEANFIB,SIGFIB,MIN_H,MAX_H,MIN_JK,MAX_JK,GAIAEDR3_SOURCE_ID,GAIAEDR3_PARALLAX,GAIAEDR3_PARALLAX_ERROR,GAIAEDR3_PMRA,GAIAEDR3_PMRA_ERROR,GAIAEDR3_PMDEC,GAIAEDR3_PMDEC_ERROR,GAIAEDR3_PHOT_G_MEAN_MAG,GAIAEDR3_PHOT_BP_MEAN_MAG,GAIAEDR3_PHOT_RP_MEAN_MAG,GAIAEDR3_DR2_RADIAL_VELOCITY,GAIAEDR3_DR2_RADIAL_VELOCITY_ERROR,GAIAEDR3_R_MED_GEO,GAIAEDR3_R_LO_GEO,GAIAEDR3_R_HI_GEO,GAIAEDR3_R_MED_PHOTOGEO,GAIAEDR3_R_LO_PHOTOGEO,GAIAEDR3_R_HI_PHOTOGEO,ASPCAP_GRID,ASPCAP_CHI2,ASPCAPFLAG,ASPCAPFLAGS,FRAC_BADPIX,FRAC_LOWSNR,FRAC_SIGSKY,EXTRATARG,MEMBERFLAG,MEMBER,TEFF,TEFF_ERR,LOGG,LOGG_ERR,M_H,M_H_ERR,ALPHA_M,ALPHA_M_ERR,VMICRO,VMACRO,VSINI,TEFF_SPEC,LOGG_SPEC,C_FE,C_FE_SPEC,C_FE_ERR,C_FE_FLAG,CI_FE,CI_FE_SPEC,CI_FE_ERR,CI_FE_FLAG,N_FE,N_FE_SPEC,N_FE_ERR,N_FE_FLAG,O_FE,O_FE_SPEC,O_FE_ERR,O_FE_FLAG,NA_FE,NA_FE_SPEC,NA_FE_ERR,NA_FE_FLAG,MG_FE,MG_FE_SPEC,MG_FE_ERR,MG_FE_FLAG,AL_FE,AL_FE_SPEC,AL_FE_ERR,AL_FE_FLAG,SI_FE,SI_FE_SPEC,SI_FE_ERR,SI_FE_FLAG,P_FE,P_FE_SPEC,P_FE_ERR,P_FE_FLAG,S_FE,S_FE_SPEC,S_FE_ERR,S_FE_FLAG,K_FE,K_FE_SPEC,K_FE_ERR,K_FE_FLAG,CA_FE,CA_FE_SPEC,CA_FE_ERR,CA_FE_FLAG,TI_FE,TI_FE_SPEC,TI_FE_ERR,TI_FE_FLAG,TIII_FE,TIII_FE_SPEC,TIII_FE_ERR,TIII_FE_FLAG,V_FE,V_FE_SPEC,V_FE_ERR,V_FE_FLAG,CR_FE,CR_FE_SPEC,CR_FE_ERR,CR_FE_FLAG,MN_FE,MN_FE_SPEC,MN_FE_ERR,MN_FE_FLAG,FE_H,FE_H_SPEC,FE_H_ERR,FE_H_FLAG,CO_FE,CO_FE_SPEC,CO_FE_ERR,CO_FE_FLAG,NI_FE,NI_FE_SPEC,NI_FE_ERR,NI_FE_FLAG,CU_FE,CU_FE_SPEC,CU_FE_ERR,CU_FE_FLAG,CE_FE,CE_FE_SPEC,CE_FE_ERR,CE_FE_FLAG
0,b'2M21342357+1215247',b'apo25m',b'M15',b'none',323.598225,12.256884,65.872040,-28.084396,7.054,0.024,6.898,0.024,6.817,0.020,0.026954,b'SFD',0.078948,0.089252,-2147483584,-2147483136,-2147483584,-2147483136,0,0,"b'APOGEE_NO_DERED,APOGEE_TELLURICAPOGEE2_NO_DE...","b'apogee-marvels,apogee,apogee2-ma'","b'apogee-marvels,apogee,cluster_gc'",43,4468.503906,4633.213867,512,b'PERSIST_HIGH',0,NaN,4.361008,0.071467,0.024969,6279.080566,4.522023,-0.416282,0.0,0.0,104.313927,22.263607,18.363949,0,1,162.401016,28.927105,-9999.990234,9999.990234,-9999.990234,9999.990234,1769924725255921408,15.189039,0.027625,-67.864182,0.027827,-138.051620,0.022424,7.93377,8.18294,7.51829,3.886700,0.315147,65.700890,65.579224,65.823608,65.714142,65.579575,65.836922,b'Fd_b',5.843325,708,"b'VMICRO_WARN,N_M_WARN,STAR_WARN,COLORTE_WARN'",0.0,0.000000,0.0,5,0,NaN,5849.514648,30.970270,3.770495,0.022910,-0.128900,0.005534,0.101417,0.007837,0.346681,0.0,6.071976,5933.100098,3.8323,0.064557,0.064557,0.039481,0,0.094450,0.094450,0.032817,0,-0.511510,-0.511510,0.312844,288,0.084793,0.080100,0.070356,0,0.032714,-0.045750,0.150773,0,-0.079752,-0.147040,0.012330,0,0.124942,0.112516,0.014039,0,0.099471,0.026328,0.012716,0,NaN,NaN,0.106314,2,0.076392,0.103460,0.045297,0,-0.133867,-0.228690,0.038239,0,-0.075809,-0.006525,0.012410,0,-0.701641,-0.654710,0.071554,256,NaN,NaN,NaN,65,-0.083359,-0.048490,0.123598,0,-0.372012,-0.209470,0.062309,0,-0.108617,-0.261300,0.013622,0,-0.111000,-0.111000,0.005964,0,NaN,NaN,NaN,64,-0.036400,-0.038750,0.011700,0,NaN,NaN,0.082007,2,NaN,NaN,NaN,64
1,b'2M05003947-7037264',b'lco25m',b'LMC_VdS',b'none',75.164470,-70.624023,282.031915,-34.568235,6.972,0.020,6.835,0.026,6.776,0.024,0.041975,b'SFD',NaN,0.138991,0,0,-2147483584,-2147483136,0,0,"b'APOGEE2_NO_DERED,APOGEE2_TELLURIC'",b'apogee2s',b'magclouds',24,4154.428711,4314.879883,131072,b'SUSPECT_BROAD_LINES',131072,b'SUSPECT_BROAD_LINES',7.172697,10.574690,0.105563,7442.392090,4.355007,-0.851826,0.0,0.0,118.781197,841.959900,720.255676,0,1,185.003113,32.352566,-9999.990234,9999.990234,-9999.990234,9999.990234,4655052273182210688,8.144106,0.018178,-6.905483,0.0

In [120]:
apogee = dfq[['APOGEE_ID', 'TEFF', 'TEFF_ERR', 'LOGG',
       'LOGG_ERR', 'M_H', 'M_H_ERR', 'FE_H', 'FE_H_ERR', 
       'VSINI', 'C_FE', 'C_FE_ERR', 'CI_FE', 'CI_FE_ERR','N_FE', 'N_FE_ERR', 'O_FE', 'O_FE_ERR', 
       'NA_FE', 'NA_FE_ERR', 'MG_FE', 'MG_FE_ERR',
       'AL_FE', 'AL_FE_ERR', 'SI_FE', 'SI_FE_ERR', 'S_FE', 'S_FE_ERR', 
       'K_FE', 'K_FE_ERR', 'CA_FE', 'CA_FE_ERR', 'TI_FE', 'TI_FE_ERR',
       'GAIAEDR3_SOURCE_ID']].copy()

In [121]:
apogee['GAIAEDR3_SOURCE_ID']

0         1769924725255921408
1         4655052273182210688
2         1262018454225426432
3         4419921058716499968
4         3974613764225607936
                 ...         
422505     335835042043520896
422506    3951123110615330048
422507    3665582552221640448
422508    6727062075794237056
422509    4280758002292391040
Name: GAIAEDR3_SOURCE_ID, Length: 422510, dtype: int64

In [122]:
column_mapping = {
    "GAIAEDR3_SOURCE_ID": "GaiaDR3",
    "TEFF":"apogee_teff",
    "TEFF_ERR":"apogee_e_teff",
    "LOGG":"apogee_logg",
    "LOGG_ERR":"apogee_e_logg",
    "FE_H":"apogee_fe_h",
    "FE_H_ERR":"apogee_e_fe_h",
    "M_H":"apogee_m_h",
    "M_H_ERR":"apogee_e_m_h",
    'C_FE': 'apogee_c_fe',
    'C_FE_ERR': 'apogee_e_c_fe',
    'CI_FE': 'apogee_ci_fe',
    'CI_FE_ERR': 'apogee_e_ci_fe',
    'N_FE': 'apogee_n_fe',
    'N_FE_ERR': 'apogee_e_n_fe',
    'O_FE': 'apogee_o_fe',
    'O_FE_ERR': 'apogee_e_o_fe',
    'NA_FE': 'apogee_na_fe',
    'NA_FE_ERR': 'apogee_e_na_fe',
    'MG_FE': 'apogee_mg_fe',
    'MG_FE_ERR': 'apogee_e_mg_fe',
    'AL_FE': 'apogee_al_fe',
    'AL_FE_ERR': 'apogee_e_al_fe',
    'SI_FE': 'apogee_si_fe',
    'SI_FE_ERR': 'apogee_e_si_fe',
    'S_FE': 'apogee_s_fe',
    'S_FE_ERR': 'apogee_e_s_fe',
    'K_FE': 'apogee_k_fe',
    'K_FE_ERR': 'apogee_e_k_fe',
    'CA_FE': 'apogee_ca_fe',
    'CA_FE_ERR': 'apogee_e_ca_fe',
    'TI_FE': 'apogee_ti_fe',
    'TI_FE_ERR': 'apogee_e_ti_fe'
}

In [123]:
apogee = apogee.rename(columns=column_mapping)

In [124]:
apogee['GaiaDR3']

0         1769924725255921408
1         4655052273182210688
2         1262018454225426432
3         4419921058716499968
4         3974613764225607936
                 ...         
422505     335835042043520896
422506    3951123110615330048
422507    3665582552221640448
422508    6727062075794237056
422509    4280758002292391040
Name: GaiaDR3, Length: 422510, dtype: int64

In [125]:
apogee.dropna(inplace=True)
apogee.reset_index(inplace=True,drop=True)

# Cast names as strings
apogee['APOGEE_ID'] = apogee['APOGEE_ID'].str.decode('utf-8')
apogee['GaiaDR3'] = apogee['GaiaDR3'].astype('|S').str.decode('utf-8')

In [128]:
apogee

,APOGEE_ID,apogee_teff,apogee_e_teff,apogee_logg,apogee_e_logg,apogee_m_h,apogee_e_m_h,apogee_fe_h,apogee_e_fe_h,VSINI,apogee_c_fe,apogee_e_c_fe,apogee_ci_fe,apogee_e_ci_fe,apogee_n_fe,apogee_e_n_fe,apogee_o_fe,apogee_e_o_fe,apogee_na_fe,apogee_e_na_fe,apogee_mg_fe,apogee_e_mg_fe,apogee_al_fe,apogee_e_al_fe,apogee_si_fe,apogee_e_si_fe,apogee_s_fe,apogee_e_s_fe,apogee_k_fe,apogee_e_k_fe,apogee_ca_fe,apogee_e_ca_fe,apogee_ti_fe,apogee_e_ti_fe,GaiaDR3
0,2M21342357+1215247,5849.514648,30.970270,3.770495,0.022910,-0.128900,0.005534,-0.111000,0.005964,6.071976,0.064557,0.039481,0.094450,0.032817,-0.511510,0.312844,0.084793,0.070356,0.032714,0.150773,-0.079752,0.012330,0.124942,0.014039,0.099471,0.012716,0.076392,0.045297,-0.133867,0.038239,-0.075809,0.012410,-0.701641,0.071554,1769924725255921408
1,2M05003947-7037264,6564.755371,61.669910,3.845554,0.030657,-0.349520,0.009302,-0.313090,0.006454,63.562359,0.139080,0.099530,0.307710,0.046610,1.127270,0.138541,-0.247777,0.259943,0.477625,0.340382,-0.180182,0.015494,0.208126,0.013826,0.051716,0.016442,0.289182,0.051560,-2.045987,0.061420,-0.305124,0.017867,-0.304261,0.160484,4655052273182210688
2,2M15044648+2224548,6358.979492,45.185886,4.373098,0.023858,-0.090514,0.005366,-0.090422,0.005243,16.500608,-0.028151,0.061618,-0.017858,0.036166,-0.417422,0.813836,0.059653,0.150990,-0.011834,0.202246,-0.111833,0.012223,-0.011692,0.012889,0.027519,0.013124,0.001648,0.042893,-0.050055,0.047362,-0.023118,0.012790,0.047965,0.109109,1262018454225426432
3,2M15145309+0116160,6028.295410,37.779453,3.719655,0.025628,-0.230090,0.005871,-0.235660,0.006354,30.095392,0.058884,0.052171,0.096636,0.036956,-0.471330,0.530466,0.072059,0.101356,-0.015926,0.197697,-0.038542,0.012984,0.060456,0.014234,0.079937,0.013500,0.103832,0.048252,0.161173,0.044318,-0.130904,0.013855,-0.534911,0.090714,4419921058716499968
4,2M11540771+1810106,6159.069824,39.814457,4.018009,0.024246,-0.143130,0.005276,-0.144170,0.005715,9.559826,0.036753,0.053812,0.064964,0.035747,0.245650,0.117719,0.091350,0.115468,-0.108326,0.190419,-0.069701,0.012364,0.128448,0.013521,0.115460,0.013052,0.057421,0.044984,0.111743,0.044594,-0.021926,0.012940,-0.386411,0.095177,3974613764225607936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123491,2M06493289+4451281,5326.513184,28.129629,4.562014,0.031502,-0.210270,0.009559,-0.201100,0.010377,1.508551,-0.016252,0.038727,0.024735,0.047037,0.000437,0.043332,0.117613,0.055098,0.216660,0.171852,0.113289,0.018623,0.088126,0.026933,0.101070,0.020299,0.112943,0.075651,-0.023427,0.045214,0.033956,0.020765,-0.113343,0.069274,953995356945186816
123492,2M17064449-2627512,4934.586914,15.956956,3.274518,0.036689,-0.463390,0.010474,-0.468400,0.011877,2.377442,0.112030,0.029201,0.167820,0.047667,-0.132590,0.031952,0.201753,0.039529,-0.013888,0.142372,0.299303,0.019738,0.242374,0.034492,0.218241,0.024448,0.127571,0.077389,0.322319,0.061831,0.224355,0.027758,0.174833,0.040051,4111930534526238208
123493,2M14441339+0349543,6072.845703,44.183178,4.398261,0.031175,0.030101,0.006634,0.036799,0.007263,4.466013,-0.079872,0.060308,-0.080562,0.047331,-0.488538,0.995964,-0.050088,0.143815,-0.492695,0.203038,-0.073509,0.016165,0.086447,0.022351,0.062936,0.019149,0.012083,0.061438,0.042978,0.055393,0.044248,0.017984,-0.126231,0.111408,1158562033617449088
123494,2M04484786-6146021,5429.867188,22.949366,4.459354,0.021056,0.261150,0.005932,0.264310,0.006403,1.503038,-0.033268,0.025189,-0.054280,0.033183,0.217620,0.023849,-0.034481,0.042145,0.239934,0.090912,0.028309,0.014095,0.137936,0.021615,0.080941,0.015927,-0.141907,0.052441,-0.186798,0.034635,-0.045154,0.013207,-0.069428,0.051530,4665556010811449984


In [126]:
# Merge with K2 data
k2_updated = k2_updated.merge(apogee,on='GaiaDR3',how='left')
k2_updated

,EPIC_ID,Campaign_x,CDPP1,CDPP1_5,CDPP2,CDPP2_5,CDPP3,CDPP4,CDPP5,CDPP6,CDPP7,CDPP8,CDPP9,CDPP10,SpT,RUWE_x,Ncomp,Rad_x,U_Rad,u_Rad,Teff_x,U_Teff,logg_x,U_logg,Mass_x,U_Mass,FeH,U_FeH,limb1,limb2,d_mag,Campaign_y,GaiaDR3,2MASS,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Dist,e_Dist,E_Dist,Teff_y,e_Teff,logg_y,e_logg,feh,e_feh,Lum,e_Lum,Rad_y,e_Rad,Mass_y,e_Mass,RUWE_y,non_single_star,Gmag,e_Gmag,BPmag,e_BPmag,RPmag,e_RPmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,BP-RP,BP-G,G-RP,RP-J,J-H,H-K,extinctionJ,extinctionH,extinctionK,absJ,e_absJ,absH,e_absH,absK,e_absK,BC,e_BC,APOGEE_ID,apogee_teff,apogee_e_teff,apogee_logg,apogee_e_logg,apogee_m_h,apogee_e_m_h,apogee_fe_h,apogee_e_fe_h,VSINI,apogee_c_fe,apogee_e_c_fe,apogee_ci_fe,apogee_e_ci_fe,apogee_n_fe,apogee_e_n_fe,apogee_o_fe,apogee_e_o_fe,apogee_na_fe,apogee_e_na_fe,apogee_mg_fe,apogee_e_mg_fe,apogee_al_fe,apogee_e_al_fe,apogee_si_fe,apogee_e_si_fe,apogee_s_fe,apogee_e_s_fe,apogee_k_fe,apogee_e_k_fe,apogee_ca_fe,apogee_e_ca_fe,apogee_ti_fe,apogee_e_ti_fe
0,201121245,1,257.151,236.598,226.563,209.841,191.452,174.727,171.493,155.131,149.860,144.238,141.405,133.911,G5,1.174929,0.0,4.415,0.348,0.329,4814,138,3.283,0.150,1.373,0.610,-0.100,0.235,0.6212,0.1129,0.0,1.0,3.5987014932879324e+18,2MASS J11471875-0600134,176.828104,0.433259,-6.003704,0.243416,520.059143,7.278503,6.073486,4902.0,143.0,3.416,0.066,-0.257,0.217,9.064502,0.684049,4.177,0.245,1.661,0.134,1.175,0.0,11.102720,0.002764,11.604373,0.002830,10.439149,0.003798,9.620,0.026,9.100,0.023,8.981,0.019,1.165,0.502,0.664,0.819,0.520,0.119,0.000,0.000,0.000,1.041,0.040,0.521,0.036,0.400,0.033,1.942,0.078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201122454,1,268.578,248.491,225.499,214.708,198.934,183.430,183.657,170.828,173.404,180.913,183.539,172.220,K7,7.138988,0.0,0.950,0.078,0.071,4046,138,4.532,0.150,1.123,0.521,-0.161,0.235,0.4639,0.2589,0.0,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201122521,1,486.386,450.572,368.257,355.254,328.753,295.503,264.617,253.877,242.677,221.716,213.211,211.158,G8,0.947940,0.0,2.522,0.287,0.247,5144,138,3.652,0.150,1.052,0.511,-0.375,0.235,0.4664,0.2249,0.0,1.0,3.598719154193455e+18,2MASS J11472021-0558179,176.834212,0.515459,-5.971652,0.287594,1847.470830,93.061530,97.159910,5094.0,149.0,3.784,0.094,-0.288,0.217,3.620656,0.499565,2.453,0.252,1.332,0.113,0.948,0.0,14.746737,0.002763,15.206782,0.003263,14.127944,0.003942,13.402,0.026,12.901,0.027,12.872,0.035,1.078,0.460,0.619,0.726,0.501,0.029,0.000,0.000,0.000,2.078,0.113,1.575,0.113,1.546,0.115,1.833,0.078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201123619,1,639.118,587.075,533.719,479.562,471.031,425.428,373.952,361.346,343.016,318.207,315.377,314.689,M1,1.127277,0.0,0.528,0.017,0.017,3676,138,4.713,0.043,0.525,0.042,-0.038,0.235,0.4651,0.2700,0.0,1.0,3.598719497790839e+18,2MASS J11471186-0556382,176.799379,0.820013,-5.943897,0.406311,200.859985,1.906356,1.719025,3729.0,109.0,4.758,0.031,-0.633,0.217,0.047647,0.002202,0.500,0.015,0.519,0.019,1.127,0.0,15.143640,0.002773,16.153950,0.003832,14.147896,0.003945,12.855,0.024,12.285,0.023,12.060,0.024,2.006,1.010,0.996,1.293,0.570,0.225,0.000,0.000,0.000,6.337,0.031,5.771,0.031,5.546,0.030,2.505,0.041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201124136,1,102.980,96.421,87.393,88.527,81.895,78.648,71.903,72.715,72.086,71.319,68.591,72.448,K7,10.175707,0.0,0.602,0.046,0.041,4028,138,4.638,0.150,0.576,0.260,-0.475,0.235,0.356

In [129]:
k2_updated['APOGEE_ID'].notna().sum()

0

In [59]:
apogee['GaiaDR3']

Series([], Name: GaiaDR3, dtype: object)

In [ ]:
dfq

In [43]:
dfq['GAIAEDR3_SOURCE_ID']

0         1769924725255921408
1         4655052273182210688
2         1262018454225426432
3         4419921058716499968
4         3974613764225607936
                 ...         
422505     335835042043520896
422506    3951123110615330048
422507    3665582552221640448
422508    6727062075794237056
422509    4280758002292391040
Name: GAIAEDR3_SOURCE_ID, Length: 422510, dtype: int64

In [42]:
apogee.columns

Index(['APOGEE_ID', 'apogee_teff', 'apogee_e_teff', 'apogee_logg',
       'apogee_e_logg', 'apogee_m_h', 'apogee_e_m_h', 'apogee_fe_h',
       'apogee_e_fe_h', 'VSINI', 'apogee_c_fe', 'apogee_e_c_fe',
       'apogee_ci_fe', 'apogee_e_ci_fe', 'apogee_n_fe', 'apogee_e_n_fe',
       'apogee_o_fe', 'apogee_e_o_fe', 'apogee_na_fe', 'apogee_e_na_fe',
       'apogee_mg_fe', 'apogee_e_mg_fe', 'apogee_al_fe', 'apogee_e_al_fe',
       'apogee_si_fe', 'apogee_e_si_fe', 'apogee_p_fe', 'apogee_e_p_fe',
       'apogee_s_fe', 'apogee_e_s_fe', 'apogee_k_fe', 'apogee_e_k_fe',
       'apogee_ca_fe', 'apogee_e_ca_fe', 'apogee_ti_fe', 'apogee_e_ti_fe',
       'GaiaDR3'],
      dtype='object')

In [121]:
# Read in GALAH DR3 data
dr3_main = Table.read('https://cloud.datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/GALAH_DR3_main_allstar_v2.fits', format='fits', hdu=1)
dr3 = np.array(dr3_main['star_id', 'sobject_id', 'dr2_source_id', 'dr3_source_id', 'survey_name', 'field_id', 'flag_repeat', 'wg4_field', 'wg4_pipeline', 'flag_sp', 'teff', 'e_teff', 'irfm_teff', 'irfm_ebv', 'irfm_ebv_ref', 'logg', 'e_logg', 'fe_h', 'e_fe_h', 'flag_fe_h', 'fe_h_atmo', 'vmic', 'vbroad', 'e_vbroad', 'chi2_sp', 'alpha_fe', 'e_alpha_fe', 'nr_alpha_fe', 'flag_alpha_fe', 'flux_A_Fe', 'chi_A_Fe', 'Li_fe', 'e_Li_fe', 'nr_Li_fe', 'flag_Li_fe', 'C_fe', 'e_C_fe', 'nr_C_fe', 'flag_C_fe', 'O_fe', 'e_O_fe', 'nr_O_fe', 'flag_O_fe', 'Na_fe', 'e_Na_fe', 'nr_Na_fe', 'flag_Na_fe', 'Mg_fe', 'e_Mg_fe', 'nr_Mg_fe', 'flag_Mg_fe', 'Al_fe', 'e_Al_fe', 'nr_Al_fe', 'flag_Al_fe', 'Si_fe', 'e_Si_fe', 'nr_Si_fe', 'flag_Si_fe', 'K_fe', 'e_K_fe', 'nr_K_fe', 'flag_K_fe', 'Ca_fe', 'e_Ca_fe', 'nr_Ca_fe', 'flag_Ca_fe', 'Sc_fe', 'e_Sc_fe', 'nr_Sc_fe', 'flag_Sc_fe', 'Sc2_fe', 'e_Sc2_fe', 'nr_Sc2_fe', 'flag_Sc2_fe', 'Ti_fe', 'e_Ti_fe', 'nr_Ti_fe', 'flag_Ti_fe', 'Ti2_fe', 'e_Ti2_fe', 'nr_Ti2_fe', 'flag_Ti2_fe', 'V_fe', 'e_V_fe', 'nr_V_fe', 'flag_V_fe', 'Cr_fe', 'e_Cr_fe', 'nr_Cr_fe', 'flag_Cr_fe', 'Cr2_fe', 'e_Cr2_fe', 'nr_Cr2_fe', 'flag_Cr2_fe', 'Mn_fe', 'e_Mn_fe', 'nr_Mn_fe', 'flag_Mn_fe', 'Co_fe', 'e_Co_fe', 'nr_Co_fe', 'flag_Co_fe', 'Ni_fe', 'e_Ni_fe', 'nr_Ni_fe', 'flag_Ni_fe', 'Cu_fe', 'e_Cu_fe', 'nr_Cu_fe', 'flag_Cu_fe', 'Zn_fe', 'e_Zn_fe', 'nr_Zn_fe', 'flag_Zn_fe', 'Rb_fe', 'e_Rb_fe', 'nr_Rb_fe', 'flag_Rb_fe', 'Sr_fe', 'e_Sr_fe', 'nr_Sr_fe', 'flag_Sr_fe', 'Y_fe', 'e_Y_fe', 'nr_Y_fe', 'flag_Y_fe', 'Zr_fe', 'e_Zr_fe', 'nr_Zr_fe', 'flag_Zr_fe', 'Mo_fe', 'e_Mo_fe', 'nr_Mo_fe', 'flag_Mo_fe', 'Ru_fe', 'e_Ru_fe', 'nr_Ru_fe', 'flag_Ru_fe', 'Ba_fe', 'e_Ba_fe', 'nr_Ba_fe', 'flag_Ba_fe', 'La_fe', 'e_La_fe', 'nr_La_fe', 'flag_La_fe', 'Ce_fe', 'e_Ce_fe', 'nr_Ce_fe', 'flag_Ce_fe', 'Nd_fe', 'e_Nd_fe', 'nr_Nd_fe', 'flag_Nd_fe', 'Sm_fe', 'e_Sm_fe', 'nr_Sm_fe', 'flag_Sm_fe', 'Eu_fe', 'e_Eu_fe', 'nr_Eu_fe', 'flag_Eu_fe', 'ra_dr2', 'dec_dr2', 'parallax_dr2', 'parallax_error_dr2', 'r_est_dr2', 'r_lo_dr2', 'r_hi_dr2', 'r_len_dr2', 'rv_galah', 'e_rv_galah', 'rv_gaia_dr2', 'e_rv_gaia_dr2', 'red_flag', 'ebv', 'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf', 'flag_guess', 'v_jk', 'j_m', 'j_msigcom', 'h_m', 'h_msigcom', 'ks_m', 'ks_msigcom', 'ph_qual_tmass', 'w2mpro', 'w2mpro_error', 'ph_qual_wise', 'a_ks', 'e_a_ks', 'bc_ks', 'ruwe_dr2'])
df = pd.DataFrame(dr3,columns=['star_id', 'sobject_id', 'dr2_source_id', 'dr3_source_id', 'survey_name', 'field_id', 'flag_repeat', 'wg4_field', 'wg4_pipeline', 'flag_sp', 'teff', 'e_teff', 'irfm_teff', 'irfm_ebv', 'irfm_ebv_ref', 'logg', 'e_logg', 'fe_h', 'e_fe_h', 'flag_fe_h', 'fe_h_atmo', 'vmic', 'vbroad', 'e_vbroad', 'chi2_sp', 'alpha_fe', 'e_alpha_fe', 'nr_alpha_fe', 'flag_alpha_fe', 'flux_A_Fe', 'chi_A_Fe', 'Li_fe', 'e_Li_fe', 'nr_Li_fe', 'flag_Li_fe', 'C_fe', 'e_C_fe', 'nr_C_fe', 'flag_C_fe', 'O_fe', 'e_O_fe', 'nr_O_fe', 'flag_O_fe', 'Na_fe', 'e_Na_fe', 'nr_Na_fe', 'flag_Na_fe', 'Mg_fe', 'e_Mg_fe', 'nr_Mg_fe', 'flag_Mg_fe', 'Al_fe', 'e_Al_fe', 'nr_Al_fe', 'flag_Al_fe', 'Si_fe', 'e_Si_fe', 'nr_Si_fe', 'flag_Si_fe', 'K_fe', 'e_K_fe', 'nr_K_fe', 'flag_K_fe', 'Ca_fe', 'e_Ca_fe', 'nr_Ca_fe', 'flag_Ca_fe', 'Sc_fe', 'e_Sc_fe', 'nr_Sc_fe', 'flag_Sc_fe', 'Sc2_fe', 'e_Sc2_fe', 'nr_Sc2_fe', 'flag_Sc2_fe', 'Ti_fe', 'e_Ti_fe', 'nr_Ti_fe', 'flag_Ti_fe', 'Ti2_fe', 'e_Ti2_fe', 'nr_Ti2_fe', 'flag_Ti2_fe', 'V_fe', 'e_V_fe', 'nr_V_fe', 'flag_V_fe', 'Cr_fe', 'e_Cr_fe', 'nr_Cr_fe', 'flag_Cr_fe', 'Cr2_fe', 'e_Cr2_fe', 'nr_Cr2_fe', 'flag_Cr2_fe', 'Mn_fe', 'e_Mn_fe', 'nr_Mn_fe', 'flag_Mn_fe', 'Co_fe', 'e_Co_fe', 'nr_Co_fe', 'flag_Co_fe', 'Ni_fe', 'e_Ni_fe', 'nr_Ni_fe', 'flag_Ni_fe', 'Cu_fe', 'e_Cu_fe', 'nr_Cu_fe', 'flag_Cu_fe', 'Zn_fe', 'e_Zn_fe', 'nr_Zn_fe', 'flag_Zn_fe', 'Rb_fe', 'e_Rb_fe', 'nr_Rb_fe', 'flag_Rb_fe', 'Sr_fe', 'e_Sr_fe', 'nr_Sr_fe', 'flag_Sr_fe', 'Y_fe', 'e_Y_fe', 'nr_Y_fe', 'flag_Y_fe', 'Zr_fe', 'e_Zr_fe', 'nr_Zr_fe', 'flag_Zr_fe', 'Mo_fe', 'e_Mo_fe', 'nr_Mo_fe', 'flag_Mo_fe', 'Ru_fe', 'e_Ru_fe', 'nr_Ru_fe', 'flag_Ru_fe', 'Ba_fe', 'e_Ba_fe', 'nr_Ba_fe', 'flag_Ba_fe', 'La_fe', 'e_La_fe', 'nr_La_fe', 'flag_La_fe', 'Ce_fe', 'e_Ce_fe', 'nr_Ce_fe', 'flag_Ce_fe', 'Nd_fe', 'e_Nd_fe', 'nr_Nd_fe', 'flag_Nd_fe', 'Sm_fe', 'e_Sm_fe', 'nr_Sm_fe', 'flag_Sm_fe', 'Eu_fe', 'e_Eu_fe', 'nr_Eu_fe', 'flag_Eu_fe', 'ra_dr2', 'dec_dr2', 'parallax_dr2', 'parallax_error_dr2', 'r_est_dr2', 'r_lo_dr2', 'r_hi_dr2', 'r_len_dr2', 'rv_galah', 'e_rv_galah', 'rv_gaia_dr2', 'e_rv_gaia_dr2', 'red_flag', 'ebv', 'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf', 'flag_guess', 'v_jk', 'j_m', 'j_msigcom', 'h_m', 'h_msigcom', 'ks_m', 'ks_msigcom', 'ph_qual_tmass', 'w2mpro', 'w2mpro_error', 'ph_qual_wise', 'a_ks', 'e_a_ks', 'bc_ks', 'ruwe_dr2'])

# Set quality cuts
unflagged_stellar_parameters = (dr3_main['flag_sp'] == 0)
dfc = df[df['flag_sp']==0]
dfc = dfc[(dfc['snr_c1_iraf']>20) | (dfc['snr_c2_iraf']>20) | (dfc['snr_c3_iraf']>20) | (dfc['snr_c4_iraf']>20)]
dfc.reset_index(inplace=True,drop=True)

# Keep only the columns you need
galah = dfc[['star_id', 'dr2_source_id', 'dr3_source_id', 'teff', 'e_teff', 'logg', 'e_logg', 'fe_h', 'e_fe_h']].copy()
galah = galah.rename(columns={"dr2_source_id": "GaiaDR2","dr3_source_id":"GaiaDR3","star_id": "GALAH_ID", "teff": "galah_teff", "e_teff": "galah_e_teff", "logg": "galah_logg", "e_logg": "galah_e_logg",
                     "fe_h": "galah_fe_h", "e_fe_h": "galah_e_fe_h"})
galah.dropna(inplace=True)
galah.reset_index(inplace=True,drop=True)

# Fix the utf encoding b string for the star_id
galah['GALAH_ID'] = galah['GALAH_ID'].str.decode('utf-8')
galah['GaiaDR3'] = galah['GaiaDR3'].astype('|S').str.decode('utf-8')

In [122]:
# Merge with K2 data
k2_updated = k2_updated.merge(galah,on=['GaiaDR3'],how='left')
k2_updated

,EPIC_ID,Campaign_x,CDPP1,CDPP1_5,CDPP2,CDPP2_5,CDPP3,CDPP4,CDPP5,CDPP6,...,apogee_fe_h,apogee_e_fe_h,GALAH_ID,GaiaDR2,galah_teff,galah_e_teff,galah_logg,galah_e_logg,galah_fe_h,galah_e_fe_h
0,201121245,1,257.151,236.598,226.563,209.841,191.452,174.727,171.493,155.131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201122454,1,268.578,248.491,225.499,214.708,198.934,183.430,183.657,170.828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201122521,1,486.386,450.572,368.257,355.254,328.753,295.503,264.617,253.877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201123619,1,639.118,587.075,533.719,479.562,471.031,425.428,373.952,361.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201124136,1,102.980,96.421,87.393,88.527,81.895,78.648,71.903,72.715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262792,211730267,18,208.273,206.536,177.306,175.085,170.118,140.380,153.043,160.229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262793,211830266,18,846.281,825.627,736.583,626.123,610.549,520.170,467.402,431.880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262794,211914998,18,100.992,92.874,93.439,93.967,98.588,108.393,109.383,113.219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262795,211962097,18,295.027,358.410,294.420,299.896,309.960,312.412,273.927,267.456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
galah

,GALAH_ID,GaiaDR2,GaiaDR3,galah_teff,galah_e_teff,galah_logg,galah_e_logg,galah_fe_h,galah_e_fe_h
0,03325271-6840304,4667368899326729856,4667368899326729856,4553.354492,137.060886,4.714953,0.201216,-0.335019,0.120219
1,03422255-6841522,4667324643983679744,4667324643983679744,4852.475586,172.750969,2.466885,0.223199,-1.170157,0.181902
2,03415061-6844479,4667323235234552064,4667323235234552064,4966.210449,176.989664,3.089965,0.215358,-1.205748,0.202541
3,03405609-6844483,4667320108498310656,4667320108498310656,5642.742676,116.182239,4.298538,0.187614,0.082866,0.118620
4,03403587-6844432,4667320795693075584,4667320795693075584,4940.007324,119.176491,4.590864,0.192101,0.109674,0.104857
...,...,...,...,...,...,...,...,...,...
417639,15142072-4415325,6003344796319529600,6003344796319529600,4531.215332,71.123153,2.410332,0.183002,-0.293808,0.042971
417640,15151322-4415182,6000435312445129984,6000435312445129984,6059.499512,89.921034,3.519315,0.190539,-0.045074,0.091634
417641,15162269-4414571,6000421607212226048,6000421607212226048,4132.972656,83.458481,0.959392,0.302143,-1.003942,0.065257
417642,15161847-4418028,6000420675196587392,6000420675196587392,4776.927246,98.422475,2.420797,0.220919,0.007314,0.073834


In [124]:
# Read in LAMOST DR9 AFGK data
df = pd.read_csv('https://www.lamost.org/dr9/v2.0/catdl?name=dr9_v2.0_LRS_stellar.csv.gz', compression='gzip', header=0, sep=',', dtype={'gaia_source_id': str})

# Set quality cuts
dfq = df[(df['snrg']>20) & (df['snrg']<999)]
dfq = dfq[(dfq.teff_err != -9999) | (dfq.logg_err != -9999) | (dfq.feh_err != -9999)]
dfq = dfq.sort_values(by='snrg',ascending=False)
dfq.drop_duplicates(subset='gaia_source_id',inplace=True)
dfq.reset_index(inplace=True,drop=True)

# Keep only the columns you need
lamostafgk = dfq[['obsid','gaia_source_id','subclass','teff','teff_err','logg','logg_err','feh','feh_err']].copy()
lamostafgk.rename(columns={'obsid':'LAMOST_ID','subclass':'lamost_SpT','teff':'lamost_teff','teff_err':'lamost_e_teff','logg':'lamost_logg','logg_err':'lamost_e_logg','feh':'lamost_fe_h','feh_err':'lamost_e_fe_h','gaia_source_id':'GaiaDR3'},inplace=True)

# Drop non-Gaia targets
lamostafgk = lamostafgk[lamostafgk['GaiaDR3'].str.contains("nan")==False]
lamostafgk['GaiaDR3'] = lamostafgk['GaiaDR3'].astype('|S').str.decode('utf-8')

# Merge with K2 data
k2_updated = k2_updated.merge(lamostafgk,on='GaiaDR3',how='left')
k2_updated

,EPIC_ID,Campaign_x,CDPP1,CDPP1_5,CDPP2,CDPP2_5,CDPP3,CDPP4,CDPP5,CDPP6,...,galah_fe_h,galah_e_fe_h,LAMOST_ID,lamost_SpT,lamost_teff,lamost_e_teff,lamost_logg,lamost_e_logg,lamost_fe_h,lamost_e_fe_h
0,201121245,1,257.151,236.598,226.563,209.841,191.452,174.727,171.493,155.131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201122454,1,268.578,248.491,225.499,214.708,198.934,183.430,183.657,170.828,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201122521,1,486.386,450.572,368.257,355.254,328.753,295.503,264.617,253.877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201123619,1,639.118,587.075,533.719,479.562,471.031,425.428,373.952,361.346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201124136,1,102.980,96.421,87.393,88.527,81.895,78.648,71.903,72.715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262792,211730267,18,208.273,206.536,177.306,175.085,170.118,140.380,153.043,160.229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262793,211830266,18,846.281,825.627,736.583,626.123,610.549,520.170,467.402,431.880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262794,211914998,18,100.992,92.874,93.439,93.967,98.588,108.393,109.383,113.219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262795,211962097,18,295.027,358.410,294.420,299.896,309.960,312.412,273.927,267.456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
lamostafgk

,LAMOST_ID,GaiaDR3,lamost_SpT,lamost_teff,lamost_e_teff,lamost_logg,lamost_e_logg,lamost_fe_h,lamost_e_fe_h
0,545405145,3376949338201658112,F4,6615.95,12.60,4.010,0.016,0.165,0.007
1,792501007,63735016867881216,F0,7141.12,10.16,4.106,0.013,-0.039,0.006
2,758803135,3382781766350685568,G3,5819.14,12.18,4.499,0.015,-0.107,0.007
3,486308232,3377098322027400320,A9,6954.77,8.77,4.195,0.011,-0.080,0.005
4,485102067,3375409747046193024,A1,8732.65,4.97,3.987,0.006,-0.316,0.003
...,...,...,...,...,...,...,...,...,...
3593338,842505207,2549549456568353152,K5,4276.66,63.20,4.695,0.102,-1.118,0.067
3593339,742803065,4554118754991269760,K7,3870.24,52.56,4.570,0.085,-0.489,0.056
3593340,698113008,345370831513836032,F5,6432.88,132.85,4.128,0.214,-0.216,0.141
3593341,83714058,2873733347552948608,F5,6395.02,68.98,4.133,0.111,-0.464,0.073
